In [6]:
import json
import faiss
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

In [7]:
f = open("data/data-module-2.json")
data = json.load(f)

f = open("data/questions.json")
questions = json.load(f)

data[0]

{'title': 'Pandemic',
 'text': 'A pandemic (from Greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. A widespread endemic disease with a stable number of infected people is not a pandemic. Widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.\nThroughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century. The term was not used yet but was for later pandemics including the 1918 influenza pandemic (Spanish flu). Current pandemics include COVID-19 (S

In [43]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
embeddings = model.encode([i["text"] for i in data], show_progress_bar=True)
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")
index = faiss.IndexFlatL2(embeddings.shape[1])
index = faiss.IndexIDMap(index)
index.add_with_ids(embeddings, np.array(range(0, len(embeddings))))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [79]:
def search(data, query: str, k=3):
  print(query)
  encoded_query = model.encode(list(query))
  D, I = index.search(np.array(encoded_query).astype("float32"), k)
  results = [data[_id] for _id in I[0]]
  return results


In [81]:
[i["title"] for i in search(data, questions[5], 5)]

Giant viruses classification


['Bills of mortality',
 'Viral load',
 'Event 201',
 'Superspreader',
 'Pandemic severity index']

In [83]:
from faiss import write_index

write_index(index, "data/faiss.index")